In [205]:
import pandas as pd 
from difflib import SequenceMatcher as SM
df_club=pd.read_csv("Clubs_data.csv")
df_meta=pd.read_csv("Metadata.csv")
df_organiser=pd.read_csv("Organisers_In_Fests.csv")
df_participants=pd.read_csv("Participants_In_Fests.csv")

In [206]:
def is_similarity(first, second, ratio):
    return SM(None, first, second).ratio() > ratio

In [207]:
for Index, row in df_club.iterrows():
    # print(Index, len(df_club.iloc[Index]["Name"]))
    df_club.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.64):
            df_club.at[Index, "Name"] = row2["Name"]
            

In [208]:
list_small_5 = []
for Index, row in df_club.iterrows():
    if len(df_club.iloc[Index]["Name"]) < 5:
        list_small_5.append(Index)


In [209]:
df_club = df_club.drop(df_club.index[list_small_5])

In [210]:
for Index, row in df_participants.iterrows():
    df_participants.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.59):
            df_participants.at[Index, "Name"] = row2["Name"]

In [211]:
list_small_7 = []
for Index, row in df_participants.iterrows():
    if len(df_participants.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)

In [212]:
df_participants = df_participants.drop(df_participants.index[list_small_7])

In [213]:
df_participants.head(30)

,Club_Name,Name,Event,Role
0,club_3,darien martens,club_3_event_2,Participant
1,club_2,sage schrock,club_2_event_1,Participant
2,club_3,alison mcvey,club_3_event_2,Participant
3,club_1,kiara jauregui,club_1_event_1,Participant
4,club_2,tate blum,club_2_event_3,Participant
5,club_2,louis samuels,club_2_event_2,Participant
6,club_2,marcel wenger,club_2_event_3,Participant
7,club_2,janine mccormack,club_2_event_2,Participant
8,club_3,quinton taylor,club_3_event_2,Participant
9,club_2,janine mccormack,club_2_event_1,Participant
